<a href="https://colab.research.google.com/github/luisgoni/Projetos-ADA/blob/main/projeot_final_ADA_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Quantas medalhas cada país conseguiu no total desde 1990?

2 - TOP 3 atletas que ganharam mais medalhas de ouro? TOP 3 medalhas de prata? TOP3 medalhas de bronze?

3 - Qual a lista de todas as modalidades existentes? A partir de que ano elas foram introduzidas nas olimpíadas?

4 - Quantas medalhas de ouro, prata e bronze cada país ganhou no vôlei (tanto masculino, quanto feminino)? Não é necessário mostrar países que nunca ganharam uma medalha no esporte.
5 - Qual a média de atletas por ano a partir de 1920 (separar verão de inverno).

EXTRA
6 - Proporção de homens e mulheres antes e depois de 1950 (compare).

In [2]:
import psycopg2
import csv

# Configurações de conexão com o banco de dados
DB_USER = ""
DB_PASSWORD = ""
DB_HOST = ""
DB_NAME = ""


conn = psycopg2.connect(f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD}")
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS regioes (
        id SERIAL PRIMARY KEY,
        noc VARCHAR(3),
        regiao VARCHAR(255),
        notes VARCHAR(255),
        CONSTRAINT unico_noc_region UNIQUE (noc, regiao)
    );
""")

with open('noc_regions.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        cur.execute("""
            INSERT INTO regioes (noc, regiao, notes)
            VALUES (%s, %s, %s)
            ON CONFLICT ON CONSTRAINT unico_noc_region DO NOTHING;
        """, (row[0], row[1], row[2]))

cur.execute("""
    CREATE TABLE IF NOT EXISTS times (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(50),
        noc VARCHAR(50),
        CONSTRAINT unico_team_noc UNIQUE (nome, noc)
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS atletas (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(100),
        genero CHAR(1),
        idade INTEGER,
        altura NUMERIC,
        peso NUMERIC,
        noc VARCHAR(3),
        time_id INTEGER REFERENCES times(id),
        CONSTRAINT unico_nome_noc UNIQUE (nome, noc)
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS esportes (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(50),
        temporada VARCHAR(6),
        regiao_id INTEGER REFERENCES regioes(id),
        CONSTRAINT unico_nome_temporada_regiao_id UNIQUE (nome, temporada, regiao_id)
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS modalidades (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(100),
        esportes_id INTEGER REFERENCES esportes(id),
        CONSTRAINT unico_nome_esportes_id UNIQUE (nome, esportes_id)
    );
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS medalhas (
        id SERIAL PRIMARY KEY,
        atletas_id INTEGER REFERENCES atletas(id),
        modalidades_id INTEGER REFERENCES modalidades(id),
        medalha VARCHAR(10),
        ano INTEGER,
        CONSTRAINT unico_atletas_modalidades UNIQUE (atletas_id, modalidades_id)
    );
""")

with open('athlete_events.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        if 'NA' not in [row[3], row[4], row[5]]:
            idade = int(row[3]) if row[3] != 'NA' else None

            cur.execute("""
            INSERT INTO times (nome, noc)
            VALUES (%s, %s)
            ON CONFLICT ON CONSTRAINT unico_team_noc DO NOTHING;
            """, (row[6], row[8]))

            cur.execute("""
                INSERT INTO atletas (nome, genero, idade, altura, peso, noc, time_id)
                VALUES (%s, %s, %s, %s, %s, %s, (SELECT id FROM times WHERE nome = %s AND noc = %s LIMIT 1))
                ON CONFLICT ON CONSTRAINT unico_nome_noc DO NOTHING;
            """, (row[1], row[2], idade, row[4], row[5], row[7], row[6], row[8]))

            cur.execute("""
                INSERT INTO esportes (nome, temporada, regiao_id)
                VALUES (%s, %s, (SELECT id FROM regioes WHERE noc = %s LIMIT 1))
                ON CONFLICT ON CONSTRAINT unico_nome_temporada_regiao_id DO NOTHING;
            """, (row[12], row[10], row[6]))

            cur.execute("""
                INSERT INTO modalidades (nome, esportes_id)
                VALUES (%s, (SELECT id FROM esportes WHERE nome = %s AND temporada = %s LIMIT 1))
                ON CONFLICT ON CONSTRAINT unico_nome_esportes_id DO NOTHING;
            """, (row[13], row[12], row[10]))

            cur.execute("""
                INSERT INTO medalhas (atletas_id, modalidades_id, medalha, ano)
                VALUES (
                    (SELECT id FROM atletas WHERE nome = %s AND noc = %s LIMIT 1),
                    (SELECT id FROM modalidades WHERE nome = %s AND esportes_id = (SELECT id FROM esportes WHERE nome = %s AND temporada = %s LIMIT 1) LIMIT 1),
                    %s, %s
                ) ON CONFLICT ON CONSTRAINT unico_atletas_modalidades DO NOTHING;
            """, (row[1], row[7], row[13], row[12], row[10], row[14], row[9]))

conn.commit()
cur.close()
conn.close()

## Codigo para verificação das respostas


-- 1 - Quantas medalhas cada país conseguiu no total desde 1990?

SELECT
    t.nome,
    COUNT(*) AS total_medalhas
FROM
    medalhas m
    JOIN atletas a ON m.atletas_id = a.id
    JOIN times t ON a.time_id = t.id
WHERE
    m.ano >= 1990
GROUP BY
    t.nome
ORDER BY
    total_medalhas DESC;

-- 2 - TOP 3 atletas que ganharam mais medalhas de ouro?

SELECT
    a.nome AS atleta,
    COUNT(*) AS ouro
FROM
    medalhas m
    JOIN atletas a ON m.atletas_id = a.id
WHERE
    m.medalha = 'Gold'
GROUP BY
    atleta
ORDER BY
    ouro DESC
LIMIT
    3;

	-- TOP 3 medalhas de prata?

SELECT
    a.nome AS atleta,
    COUNT(*) AS prata
FROM
    medalhas m
    JOIN atletas a ON m.atletas_id = a.id
WHERE
    m.medalha = 'Silver'
GROUP BY
    atleta
ORDER BY
    prata DESC
LIMIT
    3;

	-- TOP 3 medalhas de bronze?

SELECT
    a.nome AS atleta,
    COUNT(*) AS bronze
FROM
    medalhas m
    JOIN atletas a ON m.atletas_id = a.id
WHERE
    m.medalha = 'Bronze'
GROUP BY
    atleta
ORDER BY
    bronze DESC
LIMIT
    3;


-- 3 - Qual a lista de todas as modalidades existentes? A partir de que ano elas foram introduzidas nas olimpíadas?

SELECT
    m.nome AS modalidade,
    MIN(me.ano) AS ano_introducao
FROM
    modalidades m
    JOIN medalhas me ON m.id = me.modalidades_id
GROUP BY
    m.nome;

-- 4 - Quantas medalhas de ouro, prata e bronze cada país ganhou no vôlei (tanto masculino, quanto feminino)?
--Não é necessário mostrar países que nunca ganharam uma medalha no esporte.

SELECT
    t.nome,
    COUNT(CASE WHEN m.medalha = 'Gold' AND e.nome = 'Volleyball' THEN 1 END) AS ouro,
    COUNT(CASE WHEN m.medalha = 'Silver' AND e.nome = 'Volleyball' THEN 1 END) AS prata,
    COUNT(CASE WHEN m.medalha = 'Bronze' AND e.nome = 'Volleyball' THEN 1 END) AS bronze
FROM
    medalhas m
    JOIN atletas a ON m.atletas_id = a.id
    JOIN times t ON a.time_id = t.id
    JOIN esportes e ON m.modalidades_id = e.id
WHERE
    e.nome = 'Volleyball'
GROUP BY
    t.nome
HAVING
    COUNT(CASE WHEN m.medalha = 'Gold' AND e.nome = 'Volleyball' THEN 1 END) > 0
	OR COUNT(CASE WHEN m.medalha = 'Silver' AND e.nome = 'Volleyball' THEN 1 END) > 0
	OR COUNT(CASE WHEN m.medalha = 'Bronze' AND e.nome = 'Volleyball' THEN 1 END) > 0;

-- 5 - Qual a média de atletas por ano a partir de 1920 (separar verão de inverno).

SELECT
    e.temporada,
    subquery.ano,
    ROUND(AVG(num_atletas)::numeric, 2) AS media_atletas
FROM (
    SELECT
        COUNT(DISTINCT me.atletas_id) AS num_atletas,
		ano, modalidades_id
    FROM
        medalhas me

    GROUP BY
        ano, modalidades_id
	HAVING ano > 1920
) AS subquery
JOIN modalidades mo ON mo.id = subquery.modalidades_id
JOIN esportes e ON e.id = mo.esportes_id
GROUP BY
    e.temporada, subquery.ano
ORDER BY
    e.temporada, subquery.ano;

-- EXTRA
-- 6 - Proporção de homens e mulheres antes e depois de 1950 (compare).

WITH Contagens AS (
    SELECT
        COUNT(DISTINCT CASE WHEN genero = 'M' AND ano < 1950 THEN a.id END) AS homens_antes_1950,
        COUNT(DISTINCT CASE WHEN genero = 'F' AND ano < 1950 THEN a.id END) AS mulheres_antes_1950,
        COUNT(DISTINCT CASE WHEN genero = 'M' AND ano >= 1950 THEN a.id END) AS homens_apos_1950,
        COUNT(DISTINCT CASE WHEN genero = 'F' AND ano >= 1950 THEN a.id END) AS mulheres_apos_1950
    FROM
        atletas a
        JOIN medalhas me ON a.id = me.atletas_id
)
SELECT
    homens_antes_1950,
    mulheres_antes_1950,
    homens_apos_1950,
    mulheres_apos_1950,
    -- Proporção de homens antes de 1950
    ROUND((homens_antes_1950::numeric / (homens_antes_1950 + mulheres_antes_1950)) * 100, 2) AS proporcao_homens_antes_1950,
    -- Proporção de mulheres antes de 1950
    ROUND((mulheres_antes_1950::numeric / (homens_antes_1950 + mulheres_antes_1950)) * 100, 2) AS proporcao_mulheres_antes_1950,
    -- Proporção de homens depois de 1950
    ROUND((homens_apos_1950::numeric / (homens_apos_1950 + mulheres_apos_1950)) * 100, 2) AS proporcao_homens_apos_1950,
    -- Proporção de mulheres depois de 1950
    ROUND((mulheres_apos_1950::numeric / (homens_apos_1950 + mulheres_apos_1950)) * 100, 2) AS proporcao_mulheres_apos_1950
FROM Contagens;